# Lecture 25: Multi-layer Neural Network II

### How to train a neural net
Today we will learn how to apply the gradient descent algorithm on the regression problem for the model function $h(\mathbf{x}; W, b)$.


#### References:


* [Simple MNIST numpy from scratch](https://www.kaggle.com/scaomath/simple-mnist-numpy-from-scratch)
* [Stanford Deep Learning Tutorial in Matlab](http://ufldl.stanford.edu/tutorial/supervised/MultiLayerNeuralNetworks/)
* [3Blue1Brown's video series on Deep Learning](https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi)
* [A visual proof that neural nets can compute any function](http://neuralnetworksanddeeplearning.com/chap4.html)
* [Backpropagation and chain rule](http://colah.github.io/posts/2015-08-Backprop/)
* [Chapter 6: Multiplayer Feedforward Neural Network in *Deep Learning Book* by Goodfellow et al](http://www.deeplearningbook.org/contents/mlp.html)

## Review

<img src="https://faculty.sites.uci.edu/shuhaocao/files/2019/04/neural_net_3l.png" alt="drawing" width="600"/>

The model function $h(\mathbf{x}; W, b) = \mathbf{a}^{(3)} = f(\mathbf{z}^{(3)})$ in the network above, and
$$
\begin{aligned}
\mathbf{z}^{(l+1)} &= W^{(l)} \mathbf{a}^{(l)} + \mathbf{b}^{(l)}   \\
\mathbf{a}^{(l+1)} &= f(\mathbf{z}^{(l+1)})
\end{aligned}
$$

## Review

<img src="https://faculty.sites.uci.edu/shuhaocao/files/2019/04/neural_net_3l.png" alt="drawing" width="600"/>

The loss function 
$$
J(W,b; X, \mathbf{y})
= \frac{1}{N} \sum_{m=1}^N \left( \frac{1}{2} \left\| h(\mathbf{x}^{(m)}; W,b) - y^{(i)} \right\|^2 \right) + \frac{\epsilon}{2} \sum_{l=1}^{n_l-1} \; \sum_{i=1}^{s_l} \; \sum_{j=1}^{s_{l+1}} \left( w^{(l)}_{ji} \right)^2,
$$
where $n_l$ denote the number of layers, and $s_l$ denote the number of nodes in layer $l$ (not counting the bias). We can write the loss function for a single sample $(\mathbf{x}^{(i)}, y^{(i)})$ (sample loss) as 
$$
J(W,b; \mathbf{x}^{(m)}, y^{(m)}) = \frac{1}{2} \big\| h(\mathbf{x}^{(m)}; W,b) -  y^{(m)} \big\|^2,
$$
then the loss function can be written as:
$$
J(W,b; X, \mathbf{y}) = (\text{Mean of sample losses} ) + (\text{Regularization for weights} )
$$

# Training of the neural network


## Initialization
To train our neural network, we will initialize each parameter $w^{(l)}_{ij}$ and each $b^{(l)}_i$ to a small random value near zero, and then apply an optimization algorithm such as mini-batch stochastic gradient descent. Since $J(W,b)$ is a non-convex function, gradient descent-based methods without any correction (e.g., AdaBoost) is susceptible to local optima; however, in practice gradient descent usually works fairly well.

## Gradient descent

On iteration of gradient descent reads: $W = \big( w_{ij}^{(l)}\big)_{l=1}^{s_l - 1}$ (total $s_l$ layers, and $s_l-1$ set of weights between neighboring layers), and $b = \big( b_{i}^{(l)}\big)_{l=1}^{s_l - 1}$ (bias on all layers except the output layer),
> $$\begin{aligned}
w_{ij}^{(l)} &\gets w_{ij}^{(l)} - \eta \frac{\partial}{\partial w_{ij}^{(l)}} J(W,b) \\
b_{i}^{(l)} &\gets b_{i}^{(l)} - \eta \frac{\partial}{\partial b_{i}^{(l)}} J(W,b)
\end{aligned}
$$

# Backpropagation

Recall that the sample loss is $J(W,b; \mathbf{x}, y) $, we can write the partial derivatives above as:
$$
\begin{aligned}
\frac{\partial}{\partial w_{ij}^{(l)}} J(W,b) 
&=
\left\{ \frac{1}{N} \sum_{m=1}^N \frac{\partial}{\partial w_{ij}^{(l)}} J(W,b; \mathbf{x}^{(m)}, y^{(m)}) \right\} + \epsilon w_{ij}^{(l)} \\
\frac{\partial}{\partial b_{i}^{(l)}} J(W,b) &=
\frac{1}{N}\sum_{m=1}^N \frac{\partial}{\partial b_{i}^{(l)}} J(W,b; \mathbf{x}^{(m)}, y^{(m)})
\end{aligned}
$$

The intuition behind the backpropagation algorithm is as follows. Given a training example $(\mathbf{x}, y)$, we will first run a "forward pass" to compute all the activations $a_i^{(l)}$ throughout the network for each unit $i$ in every layer $l$, including the output value of the model $h(\mathbf{x};W,b)$. Then, for each unit $i$ in layer $l$, we would like to compute an "error term" $\delta^{(l)}_i$ that measures how much that node was "responsible" for any errors in our output. For an output node, we can directly measure the difference between the network's activation and the true target value, and use that to define $\delta^{(n_l)}_i$ (where layer $n_l$ is the output layer). For hidden units, we will compute $\delta^{(l)}_i$ based on a weighted average of the error terms of the units that uses $a_i^{(l)}$ as an input. In detail, here is the backpropagation algorithm:

> Step 1: Perform a forward pass, computing the activations for layers $L_2$, $L_3$, and so on up to the output layer $L_{n_l}$.
>
> Step 2: For each output unit $i$ in layer $n_l$ (the output layer), set
$$
\delta^{(n_l)}_i
:= \frac{\partial}{\partial z^{(n_l)}_i} \;J(W,b; \mathbf{x}, y) = \frac{\partial}{\partial z^{(n_l)}_i} 
\frac{1}{2} \left\|y - h(\mathbf{x};W,b)\right\|^2 = - (y - a^{(n_l)}_i) \cdot f'(z^{(n_l)}_i)
$$
>
> Step 3: For $l=n_l−1,n_l−2,n_l−3,\dots,2$ For each node $i$ in layer $l$, set
$$\delta^{(l)}_i : = \frac{\partial}{\partial z^{(l)}_i} \;J(W,b; \mathbf{x}, y) =
\frac{\partial}{\partial a^{(l)}_i} \;J(W,b; \mathbf{x}, y)  
\frac{\partial a^{(l)}_i}{\partial z^{(l)}_i}=  \left( \sum_{j=1}^{s_{l+1}} w^{(l)}_{ji} \delta^{(l+1)}_j \right) f'(z^{(l)}_i)
$$
>
> Step 4: the desired partial derivatives are computed as:
$$
\begin{aligned}
\frac{\partial}{\partial w_{ij}^{(l)}} J(W,b; \mathbf{x}, y) &= a^{(l)}_j \delta_i^{(l+1)} \\
\frac{\partial}{\partial b_{i}^{(l)}} J(W,b; \mathbf{x}, y) &= \delta_i^{(l+1)}.
\end{aligned}
$$

## Detailed derivation using chain rule

In this cell, we are going to derive the detailed formula for the derivatives of each weight in the simple 3-layer neural net. 

## Vectorized version
We will use $\ast$ to denote the element-wise product operator (which is exactly `*` operand for numpy arrays), so that if $\mathbf{a}= \mathbf{b}\ast \mathbf{c}$, then $a_i=b_i c_i$. 

Similarly to last lecture we can extend the definition of $f(\cdot)$ to apply element-wise to vectors, so that $f'([z_1, z_2, z_3]) = [f'(z_1), f'(z_2), f'(z_3)]$.

Using the notations above, the algorithm can then be rewritten:

> Step 1: Perform a forward pass, computing the activations for layers $L_2$, $L_3$, and so on up to the output layer $L_{n_l}$.
>
> Step 2: For each output unit $i$ in layer $n_l$ (the output layer), set
$$
\boldsymbol{\delta}^{(n_l)} = - (\mathbf{y} - \mathbf{a}^{(n_l)}) \ast f'(\mathbf{z}^{(n_l)})
$$
>
> Step 3: For $l=n_l−1,n_l−2,n_l−3,\dots,2$ For each node $i$ in layer $l$, set
$$
\boldsymbol{\delta}^{(l)} = \left((W^{(l)})^{\top} \boldsymbol{\delta}^{(l+1)}\right) \ast f'(\mathbf{z}^{(l)}) 
$$
>
> Step 4: the desired partial derivatives are computed as:
$$
\begin{aligned}
\nabla_{W^{(l)}} J(W,b;\mathbf{x},y) &= \boldsymbol{\delta}^{(l+1)} (\mathbf{a}^{(l)})^T, \\
\nabla_{b^{(l)}} J(W,b;\mathbf{x},y) &= \boldsymbol{\delta}^{(l+1)}.
\end{aligned}
$$

## Implementation remarks

In steps 2 and 3 above in either vectorized version or for loop version, we need to compute $f'(z^{(l)}_i)$ for each value of $i$. 
* When $f(z)$ is ReLU activation function, its derivative is just 0 when its input is less than zero and 1 when its input is greater than or equal to zero.
* When $f(z)$ is the sigmoid activation function, we would already have $a^{(l)}_i$ stored away from the forward pass through the network. Thus, using the expression that we worked out earlier for $f′(z)$, we can compute this as $f'(z^{(l)}_i) = a^{(l)}_i (1- a^{(l)}_i)$.

Finally, we are ready to describe the gradient descent algorithm for all samples. In the algorithm below, 
* $\Delta W^{(l)}$ is a matrix of the same dimension as $W^{(l)}$. Recall that $W^{(l)}$ is the matrix of weights mapping the layer $l$ activation $\mathbf{a}^{(l)}$ to match the size of layer $(l+1)$), 
* $\Delta \mathbf{b}^{(l)}$ is a vector (of the same dimension as $\mathbf{b}^{(l)}$). 

One iteration of gradient descent is then as follows:
> Step 1: $\Delta W^{(l)} := 0$ and $\Delta \mathbf{b}^{(l)} := 0$.
>
> Step 2: For $m=1$ to N, use backpropagation to compute $\nabla_{W^{(l)}} J(W,b;\mathbf{x}^{(k)},y^{(k)})$ and $\nabla_{\mathbf{b}^{(l)}} J(W,b;\mathbf{x}^{(k)},y^{(k)})$ for $k$-th sample, then
$$
\begin{aligned}
\Delta W^{(l)} & \gets \Delta W^{(l)} + \nabla_{W^{(l)}} J(W,b;\mathbf{x}^{(k)},y^{(k)})
\\
\Delta \mathbf{b}^{(l)} & \gets \Delta \mathbf{b}^{(l)} + \nabla_{b^{(l)}} J(W,b;\mathbf{x}^{(k)},y^{(k)})
\end{aligned}
$$
>
> Step 3: update the weights and biases
$$
\begin{aligned}
W^{(l)} &= W^{(l)} - \eta \left\{ \left(\frac{1}{N} \Delta W^{(l)} \right) + \epsilon W^{(l)}\right\} \\
\mathbf{b}^{(l)} &= \mathbf{b}^{(l)} - \eta \left\{\frac{1}{N} \Delta \mathbf{b}^{(l)}\right\}
\end{aligned}
$$


# Numpy implementation from scratch

Below is the implementation for the least-square loss. It unfortunately does not work for the classification problem.

In [1]:
import numpy as np

In [2]:
def relu(x):
    return x*(x>0)

def relu_prime(x):
    return 1.0*(x>0)

In [70]:
# W[0].shape = (n_features, 256), W[0] maps the input layer to the hidden layer
# W[1].shape = (256, n_target), W[1] maps the output from the hidden layer to the output layer
# b is the bias in each layer, it is also a list so that
# b[0].shape = (256,) and b[0] is the bias in the input layer
# b[1].shape = (n_target,) and b[1] is the bias in the hidden layer
def h(X,W,b):
    # layer 1 = input layer
    a1 = X
    # layer 1 (input layer) -> layer 2 (hidden layer)
    z2 = np.matmul(X, W[0]) + b[0]
    # layer 2 activation
    a2 = relu(z2)
    # layer 2 (hidden layer) -> layer 3 (output layer)
    z3 = np.matmul(a2, W[1]) + b[1]
    # output layer does not need activation for regression
    output = z3
    return output

eps = 1e-4
def loss(W,b,X,y):
    '''
    means square error loss
    '''
    residual = h(X,W,b) - y
    regularization = eps*(np.sum(W[1]**2) + np.sum(W[0]**2))
    return np.mean(residual**2) + regularization

def backprop(W,b,X,y):
    '''
    Step 1: forward pass
    Step 2: backprop
    '''
    N = X.shape[0]
    # layer 1 = input layer
    a1 = X
    # layer 1 (input layer) -> layer 2 (hidden layer)
    z2 = np.matmul(X, W[0]) + b[0]
    # layer 2 activation
    a2 = relu(z2)
    # layer 2 (hidden layer) -> layer 3 (output layer)
    z3 = np.matmul(a2, W[1]) + b[1]
    # output layer activation
    output = z3
    
    # layer 3's derivative
    delta3 = -(y - output)
    # layer 2's derivative
    delta2 = np.matmul(delta3, W[1].T)*relu_prime(z2)
    # no derivative for layer 1
    
    dW = [np.matmul(a1.T,delta2)/N + eps*W[0], np.matmul(a2.T,delta3)/N + eps*W[1]]
    db = [np.mean(delta2, axis=0), np.mean(delta3, axis=0)]
    # dW[0] is W[0]'s derivative, and dW[1] is W[1]'s derivative; similar for db
    return dW, db

In [71]:
# artificial generating data
# N is the sample size (or current mini-batch size); 
# D_in is input dimension;
# N_H is hidden dimension; 
# D_out is output dimension.
N, D_in, N_H, D_out = 10000, 1000, 256, 10
X = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

In [75]:
eta = 1e-4# step size (learning rate)
num_steps = 20

# initialization
W = [np.random.randn(D_in, N_H), np.random.randn(N_H,D_out)]
b = [np.random.randn(N_H), np.random.randn(D_out)]

In [76]:
loss(W,b,X,y)

137163.9663086516

In [ ]:
loss_at_eachstep = np.zeros(num_steps) # record the change of the loss function

for i in range(num_steps):
    loss_at_eachstep[i] = loss(W,b,X,y)
    dW, db = backprop(W,b,X,y)
    W[0] -= eta * dW[0]
    W[1] -= eta * dW[1]
    b[0] -= eta * db[0]
    b[1] -= eta * db[1]
    if i % 5 == 0:
        print("loss after", i+1, "iterations is: ", loss_at_eachstep[i])
        print("R squared", i+1, "iterations is: ", 1 - loss_at_eachstep[i]/(np.sum((y- np.mean(y))**2)))

# Exercise by yourself (very hard)

The softmax regression alone did not fit the MNIST training set well, due to lack of parameters. In the neural network version, the output unit of is identical to the softmax regression function we have seen in leture of softmax regression. As a result, the cost function is nearly identical to the softmax regression cost function. Note that instead of making predictions from the input data $\mathbf{x}$ the softmax function takes as input the final hidden layer of the neural network $h(\mathbf{x}; W, b)$. The loss function is thus,
$$
J(\theta) = - \frac{1}{N}\left\{ \sum_{i=1}^{N} \sum_{k=1}^{K}  1\left\{y^{(i)} = k\right\} 
\log \frac{\exp\big(\theta^{(k)\top} h(\mathbf{x}^{(i)}; W, b) \big)}
{\sum_{j=1}^K \exp\big(\theta^{(j)\top} h(\mathbf{x}^{(i)}; W, b) \big)}\right\}.
$$
Note since the final hidden layer of the neural network has size `(256,)`, thus $\theta$ should be of dimension `(256,10)`. The $\delta^{(n_l)}$ are different now:
$$
\delta^{(n_l)} = - \frac{1}{N}\sum_{i=1}^{N} \Big\{  1\{ y^{(i)} = k\}  - P(y^{(i)} = k | x^{(i)}; \theta)  \Big\} 
$$
However, if we still use ReLU activation for the previous layers, the $\delta^{(l)}$s for $l=n_l−1,n_l−2,n_l−3,\dots,2$ remain unchanged.

* Train and test various network architectures. You should be able to achieve 100% training set accuracy with a single hidden layer of 256 hidden units. Because the network has many parameters, there is a danger of overfitting. Experiment with layer size, number of hidden layers (less than or equal to 3), and regularization penalty to understand what types of architectures perform best. Can you find a network with multiple hidden layers which outperforms your best single hidden layer architecture?
* (Optional) Read the code and algorithms in [Simple MNIST numpy from scratch](https://www.kaggle.com/scaomath/simple-mnist-numpy-from-scratch).

In [ ]:
# load data
X = np.load('mnist_train.npz')['X']/255
y = np.load('mnist_train.npz')['y'].astype(int)

In [ ]:
# take just the first 10k sample and change the label to its one-hot encoding
X = X[:10000]
y = y[:10000]